# Neural Networks 

Neural networks can be constructed using the `torch.nn` package.

Now that you had a glimpse of `autograd`, `nn` depends on autograd to define models and differentiate them. An `nn.Module` contains layers, and a method f`orward(input)` that returns the output.

For example, look at this network that classifies digit images:

![img](./content/60_min_blitz_3_NN_img1.png)

(convnet)

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`

## Define the network 

#### Let’s define this network: 

In [2]:
import torch
import torch.nn as nn  # NN module, nn by convention
import torch.nn.functional as F  # Functions, F by convention

In [3]:
class Net(nn.Module):

    def __init__(self):
        
        super(Net, self).__init__()  # Call the parent consructor to initialise stuff.
        
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel:
        # >> Conv2d(in_channels, out_channels, kernel_size[, ...])
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)  # Note now it becomes 6 "channels" input.
        
        # An affine operation: y = Wx + b
        # Linear(in_features, out_features, bias=True)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):  # x is the *input*.
        
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))  # Note the nonlinearity added here and elsewhere below - ReLU.
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # Flatten.
        x = x.view(-1, self.num_flat_features(x))
        
        # Fully connected layers at the end.
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    # backward() will be auto-generated with `autograd`.
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the **batch dimension**.
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

* Note: inherit your "net" from `nn.Module`.
* `.Conv2D()`: https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d
* `.Linear()`: https://pytorch.org/docs/stable/nn.html#torch.nn.Linear

In [5]:
net = Net()
display(net)  # Note that the connections between layers aren't explicitly defined, that is based on .forward().

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

You just have to define the `forward` function, and the `backward` function (where gradients are computed) is automatically defined for you using `autograd`. You can use any of the `Tensor` operations in the `forward` function.

**The learnable parameters of a model are returned by `net.parameters()`**

In [6]:
params = list(net.parameters())

In [11]:
print("len(params)={} \n".format(len(params)))
print(str(params)[:1000] + "\n...")

len(params)=10

[Parameter containing:
tensor([[[[-0.0199,  0.0639, -0.1107],
          [ 0.0668,  0.2990,  0.0810],
          [ 0.0042, -0.2457,  0.1999]]],


        [[[-0.0983,  0.3058,  0.0727],
          [-0.0775,  0.1638, -0.1003],
          [ 0.3184, -0.0278, -0.2887]]],


        [[[ 0.0862,  0.2799,  0.1622],
          [ 0.2039, -0.0608, -0.1061],
          [ 0.2023, -0.2501, -0.0702]]],


        [[[-0.1480,  0.1583,  0.0267],
          [ 0.2382,  0.2834, -0.0075],
          [-0.1894, -0.0008,  0.0747]]],


        [[[-0.2462, -0.1901, -0.1427],
          [-0.3212,  0.3289,  0.0872],
          [ 0.1247, -0.3025,  0.1002]]],


        [[[-0.2468, -0.2135,  0.1983],
          [-0.2266, -0.1930, -0.2596],
          [-0.0404, -0.1577,  0.1070]]]], requires_grad=True), Parameter containing:
tensor([ 0.2425,  0.3082, -0.2659, -0.1044,  0.2444, -0.2036],
       requires_grad=True), Parameter containing:
tensor([[[[-0.1117,  0.1356,  0.0394],
          [-0.0577, -0.0572,  0.0802],
  

In [13]:
# conv1's .weight
print("parameter 0 \n")
print("params[0].size()={} \n".format(params[0].size()))
print(params[0])

parameter 0 

params[0].size()=torch.Size([6, 1, 3, 3]) 

Parameter containing:
tensor([[[[-0.0199,  0.0639, -0.1107],
          [ 0.0668,  0.2990,  0.0810],
          [ 0.0042, -0.2457,  0.1999]]],


        [[[-0.0983,  0.3058,  0.0727],
          [-0.0775,  0.1638, -0.1003],
          [ 0.3184, -0.0278, -0.2887]]],


        [[[ 0.0862,  0.2799,  0.1622],
          [ 0.2039, -0.0608, -0.1061],
          [ 0.2023, -0.2501, -0.0702]]],


        [[[-0.1480,  0.1583,  0.0267],
          [ 0.2382,  0.2834, -0.0075],
          [-0.1894, -0.0008,  0.0747]]],


        [[[-0.2462, -0.1901, -0.1427],
          [-0.3212,  0.3289,  0.0872],
          [ 0.1247, -0.3025,  0.1002]]],


        [[[-0.2468, -0.2135,  0.1983],
          [-0.2266, -0.1930, -0.2596],
          [-0.0404, -0.1577,  0.1070]]]], requires_grad=True)


#### Some notes on parameters
* In the above Conv2d (conv1 layer here) case, 6 (output channels) x 1 (input channel?) x (3x3) (convolution kernels)
* **The params don't appear to be labelled by name, they just come as a list.**

### Experiment with parameter ordering ----------

In [14]:
class Net2(nn.Module):
    ### Same as Net but define layers in a different order.
    
    def __init__(self):
        
        super(Net2, self).__init__()
        
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        
        self.conv1 = nn.Conv2d(1, 6, 3)
        
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        self.conv2 = nn.Conv2d(6, 16, 3)

    def forward(self, x):
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [15]:
net2 = Net2()
display(net2)
# Notice the print order is the same as the order the layers were defined in __init__()!

Net2(
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
)

In [16]:
params2 = list(net2.parameters())

In [17]:
print("len(params2)={} \n".format(len(params2)))
print(str(params2)[:1000] + "\n...")

len(params2)=10 

[Parameter containing:
tensor([[ 0.0256,  0.0407,  0.0194,  ..., -0.0282,  0.0013, -0.0035],
        [-0.0382,  0.0208,  0.0180,  ..., -0.0363,  0.0002,  0.0352],
        [-0.0114, -0.0196, -0.0164,  ...,  0.0044, -0.0216,  0.0358],
        ...,
        [-0.0018, -0.0058,  0.0022,  ...,  0.0186, -0.0078,  0.0288],
        [-0.0286,  0.0335, -0.0334,  ...,  0.0131, -0.0256,  0.0020],
        [-0.0223,  0.0106,  0.0058,  ..., -0.0407,  0.0331,  0.0269]],
       requires_grad=True), Parameter containing:
tensor([ 0.0233,  0.0385,  0.0167,  0.0215, -0.0334,  0.0249,  0.0191, -0.0142,
         0.0210,  0.0352,  0.0107,  0.0005,  0.0191, -0.0159, -0.0095, -0.0192,
         0.0398, -0.0364,  0.0052,  0.0312,  0.0179,  0.0152, -0.0183, -0.0065,
        -0.0167, -0.0281, -0.0137, -0.0393,  0.0400,  0.0339, -0.0293,  0.0133,
        -0.0112,  0.0320, -0.0175, -0.0255, -0.0270, -0.0311, -0.0077, -0.0095,
        -0.0275,  0.0049, -0.0360,  0.0069, -0.0077,  0.0253,  0.0074,  0.0

In [19]:
# 0th parameter weight.
print("parameter 0 \n")
print("params2[0].size()={} \n".format(params2[0].size()))
print(params2[0])

# Note: now it's not conv1's but fc1's parameters in position 0!
# So .parameters() will return the weights in the layer definitin order!

parameter 0 

params2[0].size()=torch.Size([120, 576]) 

Parameter containing:
tensor([[ 0.0256,  0.0407,  0.0194,  ..., -0.0282,  0.0013, -0.0035],
        [-0.0382,  0.0208,  0.0180,  ..., -0.0363,  0.0002,  0.0352],
        [-0.0114, -0.0196, -0.0164,  ...,  0.0044, -0.0216,  0.0358],
        ...,
        [-0.0018, -0.0058,  0.0022,  ...,  0.0186, -0.0078,  0.0288],
        [-0.0286,  0.0335, -0.0334,  ...,  0.0131, -0.0256,  0.0020],
        [-0.0223,  0.0106,  0.0058,  ..., -0.0407,  0.0331,  0.0269]],
       requires_grad=True)


#### Note on `parameters`
* **`.parameters()` will return the weights in the _layer definitin order_.**

### Experiment with parameter ordering \[END\] ----------

#### Let's try a random 32x32 input.
Note: expected input size of this net (LeNet) is 32x32. To use this net on MNIST dataset, please resize the images from the dataset to 32x32.

In [20]:
_input = torch.randn(1, 1, 32, 32)
display(_input)

tensor([[[[ 0.0206,  0.3281, -1.5938,  ...,  1.8398,  0.7286, -1.2506],
          [ 0.0960, -0.8373,  0.3181,  ..., -1.8976,  0.4062, -1.4908],
          [ 1.0482, -0.4722,  0.0423,  ..., -0.2576, -0.0701,  1.7547],
          ...,
          [ 1.3112, -1.4368,  1.3746,  ..., -2.4759, -0.6301,  1.3009],
          [ 0.9194, -0.4053, -0.4947,  ..., -0.4405,  0.1686,  0.1829],
          [-0.6960,  0.3544, -0.7624,  ...,  2.3394, -1.2457, -0.2961]]]])

In [21]:
out = net(_input)
display(out)

tensor([[ 0.0826,  0.0899, -0.0101, -0.0280, -0.0894, -0.0471, -0.0544, -0.0436,
          0.0614, -0.0471]], grad_fn=<AddmmBackward>)

In [22]:
out = net.forward(_input)
display(out)

tensor([[ 0.0826,  0.0899, -0.0101, -0.0280, -0.0894, -0.0471, -0.0544, -0.0436,
          0.0614, -0.0471]], grad_fn=<AddmmBackward>)

* `net(_input)` <=> `net.forward(_input)`.
* Output is 1x10 as expected.

#### Zero the gradient buffers of all parameters and backprops with random gradients 

In [28]:
net.zero_grad()  # Sets gradients of all **model parameters** to zero.

In [29]:
# 'Gradients': torch.randn(1, 10)
gradients = torch.randn(1, 10)
out.backward(gradients)

In [31]:
# conv1's .weight - now have been updated!
print("parameter 0 (conv1's .weight) \n")
print("params[0].size()={} \n".format(params[0].size()))
print(params[0])

parameter 0 (conv1's .weight) 

params[0].size()=torch.Size([6, 1, 3, 3]) 

Parameter containing:
tensor([[[[-0.0199,  0.0639, -0.1107],
          [ 0.0668,  0.2990,  0.0810],
          [ 0.0042, -0.2457,  0.1999]]],


        [[[-0.0983,  0.3058,  0.0727],
          [-0.0775,  0.1638, -0.1003],
          [ 0.3184, -0.0278, -0.2887]]],


        [[[ 0.0862,  0.2799,  0.1622],
          [ 0.2039, -0.0608, -0.1061],
          [ 0.2023, -0.2501, -0.0702]]],


        [[[-0.1480,  0.1583,  0.0267],
          [ 0.2382,  0.2834, -0.0075],
          [-0.1894, -0.0008,  0.0747]]],


        [[[-0.2462, -0.1901, -0.1427],
          [-0.3212,  0.3289,  0.0872],
          [ 0.1247, -0.3025,  0.1002]]],


        [[[-0.2468, -0.2135,  0.1983],
          [-0.2266, -0.1930, -0.2596],
          [-0.0404, -0.1577,  0.1070]]]], requires_grad=True)


### Note

`torch.nn` only supports mini-batches. The entire `torch.nn` package only supports inputs that are a **mini-batch of samples, and not a single sample**.

For example, `nn.Conv2d` will take in a 4D Tensor of `nSamples x nChannels x Height x Width`.

**If you have a single sample, just use `input.unsqueeze(0)` to add a fake batch dimension.**

### Before proceeding further, let’s recap all the classes you’ve seen so far.

**Recap:**
* `torch.Tensor` - A multi-dimensional array with support for `autograd` operations like `backward()`. Also holds the **gradient w.r.t. the tensor**.
* `nn.Module` - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
* `nn.Parameter` - A kind of `Tensor`, that is automatically registered as a parameter when assigned as an attribute to a `Module`.
* `autograd.Function` - Implements forward and backward definitions of an `autograd` operation. Every `Tensor` operation creates at least a single `Function` node that connects to functions that created a `Tensor` and encodes its history: `MulBackward0` etc.

**At this point, we covered:**
* Defining a neural network
* Processing inputs and calling backward

**Still Left:**
* Computing the loss
* Updating the weights of the network

## Loss Function 

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the `nn` package . A simple loss is: `nn.MSELoss` which computes the mean-squared error between the input and the target.

#### For example: 

In [33]:
output = net(_input)
target = torch.randn(10)  # A dummy target, for example.
target = target.view(1, -1)  # Make it the same shape as output (a 1x (1x10), because batching(?)).
display("target", target, "output", output)

'target'

tensor([[-0.1507,  0.2656, -1.4367, -0.0110, -1.6221, -0.0103, -1.6291, -0.0072,
         -0.9954,  0.1068]])

'output'

tensor([[ 0.0826,  0.0899, -0.0101, -0.0280, -0.0894, -0.0471, -0.0544, -0.0436,
          0.0614, -0.0471]], grad_fn=<AddmmBackward>)

In [34]:
criterion = nn.MSELoss()  # Type of loss function. Meand Squared Error here.
loss = criterion(output, target)  # Loss function **calculation** itself.

In [36]:
loss

tensor(0.8093, grad_fn=<MseLossBackward>)

#### Now, if you follow `loss` in the backward direction, using its `.grad_fn` attribute, you will see a graph of computations that looks like this:

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d

-> view -> linear -> relu -> linear -> relu -> linear

-> MSELoss -> loss

So, when we call `loss.backward()`, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient.

#### For illustration, let us follow a few steps backward:

In [37]:
display(loss.grad_fn)  # .MSELoss()
display(loss.grad_fn.next_functions[0][0])  # .Linear()
display(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # .Linear()

* Note the `.grad_fn.next_functions[0][0]` call - `.next_functions[i][j]` seems to allow traversing the Function graph.

## Backprop 

To backpropagate the error all we have to do is to `loss.backward()` (see `autograd` tutorial).

**You need to clear the existing gradients though, else gradients will be accumulated to existing gradients!**

#### Now we shall call `loss.backward()`, and have a look at conv1’s bias gradients before and after the backward. 

In [38]:
net.zero_grad()  # Zeroes the gradient buffers of all parameters.

print('conv1.bias.grad before backward')
display(net.conv1.bias.grad)

loss.backward()  # Crucial line.
print('conv1.bias.grad after backward')
display(net.conv1.bias.grad)

conv1.bias.grad before backward


tensor([0., 0., 0., 0., 0., 0.])

conv1.bias.grad after backward


tensor([-0.0037,  0.0011,  0.0029,  0.0037, -0.0169,  0.0063])

#### Now, we have seen how to use loss functions.

**Read Later:**
* The neural network package contains various modules and loss functions that form the building blocks of deep neural networks. 
* A full list with documentation is [here](https://pytorch.org/docs/nn).

**The only thing left to learn is:**
* Updating the weights of the network

#### Side note - getting parameters of a layer. ----------

In [46]:
print("list(net.conv1.parameters())")
display(list(net.conv1.parameters()))

print("\n---\n")

print("net.conv1.weight")
display(net.conv1.weight)

print("\n---\n")

print("net.conv1.bias")
display(net.conv1.bias)

list(net.conv1.parameters())


[Parameter containing:
 tensor([[[[-0.0199,  0.0639, -0.1107],
           [ 0.0668,  0.2990,  0.0810],
           [ 0.0042, -0.2457,  0.1999]]],
 
 
         [[[-0.0983,  0.3058,  0.0727],
           [-0.0775,  0.1638, -0.1003],
           [ 0.3184, -0.0278, -0.2887]]],
 
 
         [[[ 0.0862,  0.2799,  0.1622],
           [ 0.2039, -0.0608, -0.1061],
           [ 0.2023, -0.2501, -0.0702]]],
 
 
         [[[-0.1480,  0.1583,  0.0267],
           [ 0.2382,  0.2834, -0.0075],
           [-0.1894, -0.0008,  0.0747]]],
 
 
         [[[-0.2462, -0.1901, -0.1427],
           [-0.3212,  0.3289,  0.0872],
           [ 0.1247, -0.3025,  0.1002]]],
 
 
         [[[-0.2468, -0.2135,  0.1983],
           [-0.2266, -0.1930, -0.2596],
           [-0.0404, -0.1577,  0.1070]]]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2425,  0.3082, -0.2659, -0.1044,  0.2444, -0.2036],
        requires_grad=True)]


---

net.conv1.weight


Parameter containing:
tensor([[[[-0.0199,  0.0639, -0.1107],
          [ 0.0668,  0.2990,  0.0810],
          [ 0.0042, -0.2457,  0.1999]]],


        [[[-0.0983,  0.3058,  0.0727],
          [-0.0775,  0.1638, -0.1003],
          [ 0.3184, -0.0278, -0.2887]]],


        [[[ 0.0862,  0.2799,  0.1622],
          [ 0.2039, -0.0608, -0.1061],
          [ 0.2023, -0.2501, -0.0702]]],


        [[[-0.1480,  0.1583,  0.0267],
          [ 0.2382,  0.2834, -0.0075],
          [-0.1894, -0.0008,  0.0747]]],


        [[[-0.2462, -0.1901, -0.1427],
          [-0.3212,  0.3289,  0.0872],
          [ 0.1247, -0.3025,  0.1002]]],


        [[[-0.2468, -0.2135,  0.1983],
          [-0.2266, -0.1930, -0.2596],
          [-0.0404, -0.1577,  0.1070]]]], requires_grad=True)


---

net.conv1.bias


Parameter containing:
tensor([ 0.2425,  0.3082, -0.2659, -0.1044,  0.2444, -0.2036],
       requires_grad=True)

* You can obtain parameters from a specific layer using `<module>.<layer>.parameters()`
* At least on convolutional layers, there exist `.weight` and `.bias` parameters, which is clearer than getting all paramers at once with `.parameters()`

#### Side note - getting parameters of a layer. \[END\] ----------

## Update the weights 

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

`weight = weight - learning_rate * gradient`

#### We can implement this using simple python code: 

In [48]:
learning_rate = 0.01

print("net.conv1.bias before weight update")
display(net.conv1.bias)

# Iterate through *all parameters of the net*:
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)  # This is SGD.
    # ^ `.sub_()` is presumably just an inplace substitution of the parameters.

print("---")
print("net.conv1.bias after weight update")
display(net.conv1.bias)

net.conv1.bias before weight update


Parameter containing:
tensor([ 0.2425,  0.3082, -0.2659, -0.1044,  0.2444, -0.2036],
       requires_grad=True)

net.conv1.bias after weight update


Parameter containing:
tensor([ 0.2425,  0.3082, -0.2659, -0.1044,  0.2446, -0.2037],
       requires_grad=True)

In [50]:
# Investigate `.data` and `grad.data`

print("type(net.conv1.bias) = \n{}".format(type(net.conv1.bias)))  # torch.nn.parameter.Parameter
print("type(net.conv1.bias.data) = \n{}".format(type(net.conv1.bias.data)))  # torch.Tensor (so .data just gets the underlying Tensor)

print("\n---\n")

print("type(net.conv1.bias.grad) = \n{}".format(type(net.conv1.bias.grad)))  # Interestingly, this is **already** a torch.Tensor
print("type(net.conv1.bias.grad.data) = \n{}".format(type(net.conv1.bias.grad.data)))  # Calling `.data` on torch.Tensor just returns the torch.Tensor itself. 

type(net.conv1.bias) = 
<class 'torch.nn.parameter.Parameter'>
type(net.conv1.bias.data) = 
<class 'torch.Tensor'>

---

type(net.conv1.bias.grad) = 
<class 'torch.Tensor'>
type(net.conv1.bias.grad.data) = 
<class 'torch.Tensor'>


##### However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
##### To enable this, we built a small package: `torch.optim` that implements all these methods. 
##### Using it is very simple: 

In [51]:
import torch.optim as optim

In [52]:
# Create your optimizer.
optimiser = optim.SGD(net.parameters(), lr=0.01)  # Seems first argument in the constructor is the net weights/params, others are optimiser hyperparameters.

In [57]:
# In your TRAINING LOOP:
# (i.e. this is a single step)

print("net.conv1.bias before weight update")
display(net.conv1.bias)

# ----------------------------------------------------------------

optimiser.zero_grad()  # Don't forget!

output = net(_input)  # Calculate the output (forward pass).

loss = criterion(output, target)  # Calculate the loss value.

loss.backward()  # Backprop the gradient from the loss.

optimiser.step()  # HERE, the optimier's predefined step is called.

# ----------------------------------------------------------------

print("---")
print("net.conv1.bias after weight update")
display(net.conv1.bias)

net.conv1.bias before weight update


Parameter containing:
tensor([ 0.2426,  0.3082, -0.2660, -0.1045,  0.2450, -0.2037],
       requires_grad=True)

---
net.conv1.bias after weight update


Parameter containing:
tensor([ 0.2427,  0.3082, -0.2660, -0.1045,  0.2452, -0.2037],
       requires_grad=True)

### Note
Observe how gradient buffers had to be manually set to zero using `optimizer.zero_grad()`. This is because gradients are accumulated as explained in Backprop section.